In [1]:
%load_ext autoreload
import datetime
# import ete3
import itertools
# import json
import logging
import math
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import typing
# import re
import xml.etree.ElementTree as ET
# import xml.dom.minidom
# import xmlschema

In [51]:
%autoreload
from maxes.xes_loader2 import XesLoader, XesLog, XesTrace, XesEvent, XES_DATE, XES_INT, XES_STRING
# from maxes.xes_file import XesFile
# from maxes.analyze_xes import AnalyzeXes
# import maxes.analyze_sequence
# import maxes.serialization.serialize
# import maxes.graphs
import maxes.utils
# import maxes.metrics.mean_levenstein_distance
import maxes.metrics.mean_levenstein_distance
from maxes.generators.xes_generator.xes_generator1 import XesGenerator1

In [3]:
maxes.utils.setup_notebook_logging() # RUN ONLY ONCE

In [ ]:
# file_path = "/vt/md/maxes/maxes/data/BPI Challenge 2013, open problems_1_all/BPI_Challenge_2013_open_problems.xes/BPI_Challenge_2013_open_problems.xes"
# file_path = "/vt/md/maxes/maxes/data/NASA Crew Exploration Vehicle (CEV) Software Event Log_1_all/data/nasa-cev-complete-single-trace.xes/nasa-cev-complete-single-trace.xes"
file_path="/vt/md/maxes/maxes/data/Conformance Checking Challenge 2019 (CCC19)_1_all/data/CCC19 - Log XES.xes"

original_log = XesLoader().load(file_path)

generator = XesGenerator1(debug=True).fit(original_log)

logging.info("Generating")
generated_log = generator.generate()

print(original_log.event_attribute_type_map)

{'concept:name': 'string', 'lifecycle:transition': 'string', 'org:resource': 'string', 'time:timestamp': 'date', 'RESOURCE': 'string', 'ROUND': 'string', 'EVENTID': 'string', 'ACTIVITY': 'string', 'STAGE': 'string', 'VIDEOSTART': 'string', 'VIDEOEND': 'string'}


In [5]:
trace_mappings = maxes.metrics.mean_levenstein_distance.mean_levenstein_distance(original_log, generated_log)
trace_mappings

[TracesPair(original_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x134fb5fd0>, generated_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x1364797f0>, score=0.7019867549668874),
 TracesPair(original_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x1343fc2c0>, generated_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x1364b21b0>, score=0.6611570247933884),
 TracesPair(original_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x13642bb90>, generated_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x103527410>, score=0.65625),
 TracesPair(original_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x1363d36e0>, generated_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence object at 0x13615fb30>, score=0.6461538461538461),
 TracesPair(original_sequence=<maxes.metrics.mean_levenstein_distance.TraceSequence

In [11]:
traces_pair = trace_mappings[0]

print(f"Original trace index: {traces_pair.original_sequence.index}")
original_trace = traces_pair.original_sequence.trace

print(f"Generated trace index: {traces_pair.generated_sequence.index}")
generated_trace = traces_pair.generated_sequence.trace

print(f"Score: {traces_pair.score}")

Original trace index: 2
Generated trace index: 1
Score: 0.75


In [14]:
series = original_log.traces[0].df["time:timestamp"]
range = series.max() - series.min()

# time_range_lookup_table = {
#     "milliseconds": 1000
#     "seconds": 1000,
# }

range.seconds

1320

In [16]:
dt = datetime.datetime(year=0, month=0, day=0)
dt.second

TypeError: function missing required argument 'year' (pos 1)

In [33]:
# class MseContext:
#     log: XesLog
#     event_attribute_type_map: dict[str, str]
#     attributes_range_lengths: dict[str, float]
#     attributes_weights: dict[str, float]

# def attribute_mean_square_error(original_log: XesLog, original_attribute_value, generated_attribute_value) -> float:
#     pass


# class AttributeTimeResolution(typing.NamedTuple):
#     attribute_name: str
#     min_value: datetime.datetime
#     max_value: datetime.datetime
#     precision: str


# def calculate_appropriate_time_resolutions(log: XesLog) -> list[AttributeTimeResolution]:
#     # Per trace
#     time_resolutions: list[AttributeTimeResolution] = []

#     for attribute_name, attribute_type in log.event_attribute_type_map.items():
#         if attribute_type != XES_DATE: continue

#         range_lengths = (
#             trace.df[attribute_name].max() - trace.df[attribute_name].min()
#             for trace in log.traces
#         )

#         biggest_range_length = max(range_lengths)


def get_attributes_weights1(original_log: XesLog) -> dict[str, float]:
    # All attributes are equally important

    event_attributes_count = len(original_log.event_attribute_type_map)
    attributes_weights = {attribute_name: 1.0 / float(event_attributes_count)
                          for attribute_name in original_log.event_attribute_type_map.keys()}

    return attributes_weights

def get_attributes_weights2(original_log: XesLog) -> dict[str, float]:
    # String attributes are less important

    attributes_weights: dict[str, float] = {}

    for attribute_name, attribute_type in original_log.event_attribute_type_map.items():
        weight = 1.0

        if attribute_type == 'str':
            weight = 0.2

        attributes_weights[attribute_name] = weight

    return maxes.utils.normalize(attributes_weights)

def calculate_attributes_range_lengths(original_log: XesLog, original_trace: XesTrace) -> dict[str, float]:
    attributes_range_lengths: dict[str, float] = {}

    for attribute_name, attribute_type in original_log.event_attribute_type_map.items():
        range_length = None

        if attribute_type == XES_INT:
            series = original_trace.df[attribute_name]
            range_length = series.max() - series.min()

        if attribute_type == XES_DATE:
            series = original_trace.df[attribute_name]
            range_length = (series.max() - series.min()).seconds

        if range_length != None:
            attributes_range_lengths[attribute_name] = range_length

    return attributes_range_lengths

def calculate_event_error(
        original_event: XesEvent,
        generated_event: XesEvent,

        event_attribute_type_map: dict[str, str],
        attributes_range_lengths: dict[str, float],
        attributes_weights: dict[str, float]
        ) -> float:
    event_error = 0

    for attribute_name, attribute_type in event_attribute_type_map.items():
        attribute_error = 0

        if attribute_type == XES_STRING:
            if original_event[attribute_name] != generated_event[attribute_name]:
                attribute_error = 1

        if attribute_type == XES_INT:
            delta = original_event[attribute_name] - generated_event[attribute_name]
            scaled_delta = delta / attributes_range_lengths[attribute_name]
            attribute_error = scaled_delta ** 2

        if attribute_type == XES_DATE:
            delta = original_event[attribute_name].second - generated_event[attribute_name].second
            scaled_delta = delta / attributes_range_lengths[attribute_name]
            attribute_error = scaled_delta ** 2

        print(f"attribute_name: {attribute_name}, attribute_error: {attribute_error}")

        # TODO: Other types

        weighted_attribute_error = attribute_error * attributes_weights[attribute_name]
        event_error += weighted_attribute_error

    return event_error

def calculate_trace_error(
        original_trace: XesTrace,
        generated_trace: XesTrace,

        event_attribute_type_map: dict[str, str],
        attributes_weights: dict[str, float],
        ) -> float:
    attributes_range_lengths = calculate_attributes_range_lengths(original_log, original_trace)

    accumulated_error = 0

    for original_event in original_trace.events:
        # TODO: Find best event by lowest error

        generated_events_candidates = [
            {
                "event": event,
                "error": calculate_event_error(
                    original_event=original_event,
                    generated_event=event,
                    event_attribute_type_map=event_attribute_type_map,
                    attributes_range_lengths=attributes_range_lengths,
                    attributes_weights=attributes_weights
                )
            }
            for event in generated_trace.events
            if event.concept_name == original_event.concept_name
        ]

        if len(generated_events_candidates) == 0:
            # print('skipped event')
            continue
        # print('used event')

        suitable_event_with_error = min(generated_events_candidates, key=lambda item: item["error"])
        event_error = suitable_event_with_error["error"]

        accumulated_error += event_error

    mean_error = accumulated_error / float(len(original_trace.events))
    return mean_error

def calculate_log_error(
        original_log: XesLog,
        traces_pairs: list[tuple[XesTrace, XesTrace]],
        attribute_weights: dict[str, float]
        ) -> float:
    accumulated_error = 0

    for original_trace, generated_trace in traces_pairs:
        trace_error = calculate_trace_error(
            original_trace,
            generated_trace,
            event_attribute_type_map=original_log.event_attribute_type_map,
            attributes_weights=attribute_weights
        )

        accumulated_error += trace_error

    mean_log_error = accumulated_error / float(len(original_log.traces))

    return mean_log_error


In [ ]:

trace_pairs = [(trace_pair.original_sequence.trace, trace_pair.generated_sequence.trace) for trace_pair in trace_mappings]
attribute_weights = get_attributes_weights1(original_log)

calculate_log_error(
    original_log,
    trace_pairs,
    attribute_weights=attribute_weights
)

In [18]:
attribute_weights = get_attributes_weights1(original_log)
attribute_weights

{'concept:name': 0.09090909090909091,
 'lifecycle:transition': 0.09090909090909091,
 'org:resource': 0.09090909090909091,
 'time:timestamp': 0.09090909090909091,
 'RESOURCE': 0.09090909090909091,
 'ROUND': 0.09090909090909091,
 'EVENTID': 0.09090909090909091,
 'ACTIVITY': 0.09090909090909091,
 'STAGE': 0.09090909090909091,
 'VIDEOSTART': 0.09090909090909091,
 'VIDEOEND': 0.09090909090909091}

In [53]:
original_trace = original_log.traces[0]

# find suitable generated trace
pair = next(pair for pair in trace_mappings if pair.original_sequence.index == 0)
generated_trace = pair.generated_sequence.trace

print(f"Generated trace index: {pair.generated_sequence.index}")

Generated trace index: 10


In [54]:
attributes_range_lengths = calculate_attributes_range_lengths(original_log, original_trace)
attributes_range_lengths

{'time:timestamp': 1320}

In [55]:
calculate_trace_error(
    original_trace,
    generated_trace,
    event_attribute_type_map=original_log.event_attribute_type_map,
    attributes_weights=attribute_weights
)

attribute_name: concept:name, attribute_error: 0
attribute_name: lifecycle:transition, attribute_error: 0
attribute_name: org:resource, attribute_error: 1
attribute_name: time:timestamp, attribute_error: 0.0
attribute_name: RESOURCE, attribute_error: 1
attribute_name: ROUND, attribute_error: 1
attribute_name: EVENTID, attribute_error: 1
attribute_name: ACTIVITY, attribute_error: 1
attribute_name: STAGE, attribute_error: 1
attribute_name: VIDEOSTART, attribute_error: 1
attribute_name: VIDEOEND, attribute_error: 1
attribute_name: concept:name, attribute_error: 0
attribute_name: lifecycle:transition, attribute_error: 1
attribute_name: org:resource, attribute_error: 1
attribute_name: time:timestamp, attribute_error: 0.0018646694214876033
attribute_name: RESOURCE, attribute_error: 0
attribute_name: ROUND, attribute_error: 0
attribute_name: EVENTID, attribute_error: 1
attribute_name: ACTIVITY, attribute_error: 1
attribute_name: STAGE, attribute_error: 1
attribute_name: VIDEOSTART, attribute_

0.3527055198554541

In [56]:
original_event = original_trace.events[0]
generated_event = generated_trace.events[0]

calculate_event_error(
    original_event=original_event,
    generated_event=generated_event,
    event_attribute_type_map=original_log.event_attribute_type_map,
    attributes_weights=attribute_weights,
    attributes_range_lengths=attributes_range_lengths
)

attribute_name: concept:name, attribute_error: 1
attribute_name: lifecycle:transition, attribute_error: 0
attribute_name: org:resource, attribute_error: 1
attribute_name: time:timestamp, attribute_error: 0.0
attribute_name: RESOURCE, attribute_error: 1
attribute_name: ROUND, attribute_error: 1
attribute_name: EVENTID, attribute_error: 1
attribute_name: ACTIVITY, attribute_error: 1
attribute_name: STAGE, attribute_error: 1
attribute_name: VIDEOSTART, attribute_error: 1
attribute_name: VIDEOEND, attribute_error: 1


0.8181818181818183

In [57]:
original_event["time:timestamp"]

datetime.datetime(2018, 10, 11, 12, 0, tzinfo=datetime.timezone.utc)

In [61]:
generated_event["time:timestamp"].astimezone(datetime.timezone.utc)

datetime.datetime(2019, 1, 20, 14, 0, 0, 240735, tzinfo=datetime.timezone.utc)

In [60]:
generated_event["time:timestamp"]

datetime.datetime(2019, 1, 20, 16, 0, 0, 240735)

In [38]:
delta = original_event["time:timestamp"].second - generated_event["time:timestamp"].second
print(delta)

scaled_delta = delta / attributes_range_lengths["time:timestamp"]
print(scaled_delta)

attribute_error = scaled_delta ** 2
print(attribute_error)

0
0.0
0.0


In [30]:
original_trace.events[0].attributes

{'concept:name': 'Hand washing',
 'lifecycle:transition': 'start',
 'org:resource': 'R_13_1C',
 'time:timestamp': datetime.datetime(2018, 10, 11, 12, 0, tzinfo=datetime.timezone.utc),
 'RESOURCE': 'R_13_1C',
 'ROUND': 'Pre',
 'EVENTID': '1539301115461',
 'ACTIVITY': 'Hand washing',
 'STAGE': 'Operator and Patient Preparation',
 'VIDEOSTART': '55',
 'VIDEOEND': '67'}

In [31]:
generated_trace.events[0].attributes

{'concept:name': 'Ultrasound configuration',
 'lifecycle:transition': 'start',
 'time:timestamp': datetime.datetime(2019, 1, 20, 16, 0, 0, 240735),
 'org:resource': 'R_33_1L',
 'RESOURCE': 'R_14_1D',
 'ROUND': 'Post',
 'EVENTID': '1548033072839',
 'ACTIVITY': 'Puncture',
 'STAGE': 'Install Catheter',
 'VIDEOSTART': '911',
 'VIDEOEND': '113'}

In [13]:
def aux_log_mse(original_log: XesLog, generated_log: XesLog) -> float:
    pass

def aux_trace_mse

{'concept:name': 'Prepare implements',
 'lifecycle:transition': 'start',
 'org:resource': 'R_13_1C',
 'time:timestamp': datetime.datetime(2019, 1, 15, 14, 0, tzinfo=datetime.timezone.utc),
 'RESOURCE': 'R_13_1C',
 'ROUND': 'Post',
 'EVENTID': '1547682577856',
 'ACTIVITY': 'Prepare implements',
 'STAGE': 'Operator and Patient Preparation',
 'VIDEOSTART': '27',
 'VIDEOEND': '44'}

In [ ]:
def mean_square_error(original_log: XesLog, generated_log: XesLog) -> float:
    pass

def find_corresponding_event(original_event: XesEvent, generated_trace: XesTrace):

    pass

def event_mean_square_error(original_event, generated_event):
    pass